<a href="https://colab.research.google.com/github/sanuthit/Risk-Based-Motor-Insurance-Premium-Calculation-System-/blob/risk-model-development/accident_risk_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Risk-Based Motor Insurance Premium + Premium Calculation** - Risk Model

In [7]:
import pandas as pd
import os

from google.colab import drive
drive.mount('/content/drive')
root = "/content/drive/MyDrive"
print("MyDrive exists:", os.path.exists(root))
print("Top folders:", os.listdir(root)[:30])
DATA_DIR = "/content/drive/MyDrive/Data/Datasets"
print("DATA_DIR exists:", os.path.exists(DATA_DIR))
print(os.listdir(DATA_DIR)[:30])

df = pd.read_csv("/content/drive/MyDrive/Data/Datasets/risk_dataset_60000_toyota_suzuki_v2_cleaned.csv", encoding="utf-8")

DATA_PATH = "/content/drive/MyDrive/Data/Datasets/risk_dataset_60000_toyota_suzuki_v2_cleaned.csv"
df = pd.read_csv(DATA_PATH)

print(df.shape)
df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MyDrive exists: True
Top folders: ['Colab Notebooks', 'Data']
DATA_DIR exists: True
['premium_dataset_60000_v3_toyota_suzuki_full.csv', 'risk_dataset_60000.csv', 'risk_dataset_60000_toyota_suzuki_v2_cleaned.csv']
(60000, 49)


,policy_id,customer_id,driver_age,driver_gender,driver_occupation,years_of_driving_experience,member_automobile_assoc_ceylon,has_previous_motor_policy,ncb_percentage,accidents_last_3_years,...,approx_market_value,sum_insured,total_claim_amount_within_1_year,hard_flag_blacklist,driver_age_band,vehicle_age_band,risk_exposure_proxy,doc_missing_score,compliance_risk_score,ncb_validity_flag
0,P000001,C00002,35,M,Accountant,17,1,1,20,1,...,9375583,7691446,0,0,35–44,13+,Low,1,1,0
1,P000002,C00003,40,M,Unemployed,16,0,0,0,0,...,8789777,8210229,0,0,35–44,13+,Low,0,1,0
2,P000003,C00004,33,F,Businessman,8,0,1,10,4,...,5143262,4628639,680769,0,25–34,13+,Low,1,0,0
3,P000004,C00005,45,F,Farmer,27,0,1,35,0,...,7518522,7142596,0,0,45–59,13+,High,0,0,0
4,P000005,C00006,51,F,Businessman,18,0,1,10,1,...,6677872,6343978,0,0,45–59,4–7,High,1,1,0


1. Define the FINAL risk feature list

In [8]:

RISK_FEATURES = [
    # Driver risk
    "driver_age",
    "driver_age_band",
    "driver_gender",
    "driver_occupation",
    "years_of_driving_experience",
    "member_automobile_assoc_ceylon",

    # Driving & claim history (inputs only)
    "has_previous_motor_policy",
    "accidents_last_3_years",
    "ncb_percentage",

    # Vehicle risk
    "vehicle_type",
    "vehicle_segment",
    "engine_capacity_cc",
    "fuel_type",
    "vehicle_age_years",
    "vehicle_age_band",
    "has_lpg_conversion",

    # Usage & exposure
    "vehicle_usage_type",
    "risk_exposure_proxy",
    "registration_district",
    "parking_type",

    # Behavioural / compliance proxy (optional but allowed)
    "doc_missing_score",
    "compliance_risk_score"
]


2. Define the target

In [9]:
TARGET = "had_claim_within_1_year"

In [10]:
df_risk = df[RISK_FEATURES + [TARGET]].copy()

print(df_risk.shape)
df_risk.head()


(60000, 23)


,driver_age,driver_age_band,driver_gender,driver_occupation,years_of_driving_experience,member_automobile_assoc_ceylon,has_previous_motor_policy,accidents_last_3_years,ncb_percentage,vehicle_type,...,vehicle_age_years,vehicle_age_band,has_lpg_conversion,vehicle_usage_type,risk_exposure_proxy,registration_district,parking_type,doc_missing_score,compliance_risk_score,had_claim_within_1_year
0,35,35–44,M,Accountant,17,1,1,1,20,Car,...,13,13+,0,Private,Low,Jaffna,Street,1,1,0
1,40,35–44,M,Unemployed,16,0,0,0,0,Car,...,22,13+,0,Private,Low,Kandy,Garage,0,1,0
2,33,25–34,F,Businessman,8,0,1,4,10,Car,...,21,13+,0,Private,Low,Colombo,Street,1,0,1
3,45,45–59,F,Farmer,27,0,1,0,35,SUV,...,14,13+,0,Hire,High,Kandy,Garage,0,0,0
4,51,45–59,F,Businessman,18,0,1,1,10,Car,...,7,4–7,0,Hire,High,Galle,Garage,1,1,0


3. Check

In [11]:
#missng values
df_risk.isnull().sum().sort_values(ascending=False)

,0
driver_age,0
driver_age_band,0
driver_gender,0
driver_occupation,0
years_of_driving_experience,0
member_automobile_assoc_ceylon,0
has_previous_motor_policy,0
accidents_last_3_years,0
ncb_percentage,0
vehicle_type,0


In [12]:
#Target balance
df_risk[TARGET].value_counts(normalize=True)

,proportion
had_claim_within_1_year,
0,0.859833
1,0.140167


# 4. Split X and y

In [40]:
X = df_risk[RISK_FEATURES]
y = df_risk[TARGET]

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (60000, 22)
y shape: (60000,)


In [42]:
from sklearn.model_selection import train_test_split

# 70% Train, 30% Temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X,
    y,
    test_size=0.30,
    stratify=y,
    random_state=42
)

# Split temp into 15% Validation, 15% Test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.50,
    stratify=y_temp,
    random_state=42
)

print("Train:", X_train.shape, y_train.shape)
print("Val:  ", X_val.shape, y_val.shape)
print("Test: ", X_test.shape, y_test.shape)


Train: (42000, 22) (42000,)
Val:   (9000, 22) (9000,)
Test:  (9000, 22) (9000,)


01. Encoding pipeline

In [43]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [44]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        # Ordinal bands
        ("ord", OrdinalEncoder(
            categories=[
                ["18-24", "25-34", "35-44", "45-59", "60+"],
                ["0-3", "4-7", "8-12", "13+"]
            ],
            handle_unknown="use_encoded_value",
            unknown_value=-1
        ), ["driver_age_band", "vehicle_age_band"]),

        # Nominal categories
        ("cat", OneHotEncoder(
            handle_unknown="ignore",
            sparse_output=False
        ), [
            "driver_gender",
            "driver_occupation",
            "vehicle_type",
            "vehicle_segment",
            "fuel_type",
            "vehicle_usage_type",
            "risk_exposure_proxy",
            "registration_district",
            "parking_type"
        ]),

        # Numeric / binary
        ("num", "passthrough", [
            "driver_age",
            "years_of_driving_experience",
            "member_automobile_assoc_ceylon",
            "has_previous_motor_policy",
            "accidents_last_3_years",
            "ncb_percentage",
            "engine_capacity_cc",
            "vehicle_age_years",
            "has_lpg_conversion",
            "doc_missing_score",
            "compliance_risk_score"
        ])
    ]
)


In [45]:
X_train_enc = preprocessor.fit_transform(X_train)
X_val_enc   = preprocessor.transform(X_val)
X_test_enc  = preprocessor.transform(X_test)

print(X_train_enc.shape)
print(X_val_enc.shape)
print(X_test_enc.shape)

(42000, 55)
(9000, 55)
(9000, 55)


# 02. Class imbalance handling

2.1 class weights

In [46]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.array([0, 1])
weights = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=y_train
)

class_weight = {0: weights[0], 1: weights[1]}
class_weight


{0: np.float64(0.5815080441946113), 1: np.float64(3.56718192627824)}